In [1]:
import pandas as pd
import numpy as np

使用线性回归方法，包括回归系数参数估计、回归显著性检验、回归系数检验，求出最优选模型；

In [2]:
df_data = pd.read_csv('./Advertising.csv')
df_data

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [3]:
corr_data = df_data.corr('pearson')
# 说明都有线性关系，或强或弱
print(corr_data.loc['sales'])

TV           0.782224
radio        0.576223
newspaper    0.228299
sales        1.000000
Name: sales, dtype: float64


In [4]:
Y = df_data.pop('sales').values
df_data.insert(0, 'bias', np.ones(df_data.shape[0]))
X = df_data.values

In [5]:
# 添加截距项
X

array([[  1. , 230.1,  37.8,  69.2],
       [  1. ,  44.5,  39.3,  45.1],
       [  1. ,  17.2,  45.9,  69.3],
       [  1. , 151.5,  41.3,  58.5],
       [  1. , 180.8,  10.8,  58.4],
       [  1. ,   8.7,  48.9,  75. ],
       [  1. ,  57.5,  32.8,  23.5],
       [  1. , 120.2,  19.6,  11.6],
       [  1. ,   8.6,   2.1,   1. ],
       [  1. , 199.8,   2.6,  21.2],
       [  1. ,  66.1,   5.8,  24.2],
       [  1. , 214.7,  24. ,   4. ],
       [  1. ,  23.8,  35.1,  65.9],
       [  1. ,  97.5,   7.6,   7.2],
       [  1. , 204.1,  32.9,  46. ],
       [  1. , 195.4,  47.7,  52.9],
       [  1. ,  67.8,  36.6, 114. ],
       [  1. , 281.4,  39.6,  55.8],
       [  1. ,  69.2,  20.5,  18.3],
       [  1. , 147.3,  23.9,  19.1],
       [  1. , 218.4,  27.7,  53.4],
       [  1. , 237.4,   5.1,  23.5],
       [  1. ,  13.2,  15.9,  49.6],
       [  1. , 228.3,  16.9,  26.2],
       [  1. ,  62.3,  12.6,  18.3],
       [  1. , 262.9,   3.5,  19.5],
       [  1. , 142.9,  29.3,  12.6],
 

In [6]:
from numpy.linalg import inv


# 参数估计
def cal_beta_hat(x, y):
    return np.dot(np.dot(inv(np.dot(x.T, x)), x.T), y)


def cal_q(x, y):
    beta = cal_beta_hat(x, y)
    temp = y - np.dot(x, beta)
    return np.dot(temp.T, temp)


def cal_sigma_hat_2(x, y):
    q = cal_q(x, y)
    return q / (x.shape[0] - x.shape[1] - 1)


def cal_u(x, y):
    # 因为Lyy = U + Q
    return y.var() * y.shape[0] - cal_q(x, y)


# 偏回归平方和:
def cal_F(indices: set, y, mode=0, additional_index=None):
    """
    获取自变量的F，用于判断是否引入/删除此自变量

    其中mode=0时代表计算当前集合的线性回归；mode=1代表逐步回归中的引入；mode=2代表逐步回归中的排除
    """
    if mode == 0:
        indices = list(indices)
        x = X[:, indices]
        return (x.shape[0] - 2) * cal_u(x, y) / cal_q(x, y)
    else:
        if mode == 1:
            new_indices = indices.copy()
            new_indices.add(additional_index)
            x = X[:, list(indices)]
            new_x = X[:, list(new_indices)]
            old_u = cal_u(x, y)
            new_u = cal_u(new_x, y)
            return (new_u - old_u) / cal_sigma_hat_2(new_x, y)
        else:
            new_indices = indices.copy()
            new_indices.remove(additional_index)
            x = X[:, list(indices)]
            new_x = X[:, list(new_indices)]
            old_u = cal_u(x, y)
            new_u = cal_u(new_x, y)
            return (old_u - new_u) / cal_sigma_hat_2(x, y)

In [7]:
from scipy.stats import f

ADDITION = 1
DELETION = 2


def stepwise_regression():
    # 逐步回归选择最佳模型
    in_alpha = 0.05
    # 引入的显著性水平应小于排除的显著性水平，否则将会死循环
    out_alpha = 0.1
    p = X.shape[1]
    n = X.shape[0]
    max_F = -1
    max_ind = -1
    # 找到最大一元回归
    for i in range(1, p):
        if (new_F := cal_F({0, i}, Y)) > max_F:
            max_F = new_F
            max_ind = i

    # 先判断这个一元是否满足显著性假设，不满足则直接失败。
    F_alpha = f.isf(in_alpha, 1, n - 2)
    assert max_F >= F_alpha, f"最大的偏F为{max_F}，而显著性水平{in_alpha}下的F_alpha(1, {n - 2})为{F_alpha}，" \
                             f"一元回归没有显著因素，回归结束"
    print(f"一元：引入变量{df_data.columns[max_ind]}")
    selected_indices = set()
    # 截距项一定要考虑
    selected_indices.add(0)
    selected_indices.add(max_ind)

    while True:
        max_F = -1
        max_ind = -1
        if len(selected_indices) == p:
            break
        # 开始前进
        for i in range(p):
            if i not in selected_indices:
                if (new_F := cal_F(selected_indices, Y, ADDITION, i)) > max_F:
                    max_F = new_F
                    max_ind = i
        selected_indices.add(max_ind)
        F_alpha = f.isf(in_alpha, 1, n - len(selected_indices) - 1)
        if max_F < F_alpha:
            print(f"最大的偏F为{max_F}，而显著性水平{in_alpha}下的F_alpha(1, {n - len(selected_indices) - 1})"
                  f"为{F_alpha}，没有其他显著因素，回归结束")
            break
        print(f"多元：引入变量{df_data.columns[max_ind]}，现在显著因素有{df_data.columns[list(selected_indices)].values}")
        # 开始后退
        out_flag = True
        while out_flag:
            out_flag = False
            F_alpha = f.isf(out_alpha, 1, n - len(selected_indices) - 1)
            for i in selected_indices:
                if i == 0:
                    continue
                tmp = cal_F(selected_indices, Y, DELETION, i)
                if tmp < F_alpha:
                    selected_indices.remove(i)
                    print(f"多元：引入变量{df_data.columns[max_ind]}后，判断偏回归平方和变化，"
                          f"删除变量{df_data.columns[i]}，现在显著因素有{df_data.columns[selected_indices].values}")
                    out_flag = True
                    break
        print(f"多元：对变量{df_data.columns[max_ind]}的后退判断已经结束，"
              f"现在显著因素有{df_data.columns[list(selected_indices)].values}")

    print(f"多元回归结束，显著因素有{df_data.columns[list(selected_indices)].values}")
    return selected_indices

efficient_attribute_indices = stepwise_regression()

一元：引入变量TV
多元：引入变量radio，现在显著因素有['bias' 'TV' 'radio']
多元：对变量radio的后退判断已经结束，现在显著因素有['bias' 'TV' 'radio']
最大的偏F为0.031068718342499267，而显著性水平0.05下的F_alpha(1, 195)为3.8895888198396573，没有其他显著因素，回归结束
多元回归结束，显著因素有['bias' 'TV' 'radio' 'newspaper']


In [13]:
# 回归效果显著性检验
n = Y.shape[0]
p = len(efficient_attribute_indices)
eff_test_f = (n - p - 1) * cal_u(X[:,list(efficient_attribute_indices)], Y) / p / cal_q(X[:,list(efficient_attribute_indices)], Y)

alpha = 0.05
F_alpha = f.isf(alpha, p, n - p - 1)
print(f"多元回归的显著性检验F值 = {eff_test_f}，而在显著性水平{alpha}的情况下F({p}, {n - p - 1})为{F_alpha}，"
      f"所以该多元回归{'没' if eff_test_f < F_alpha else ''}有显著性")

if eff_test_f >= F_alpha:
    beta = cal_beta_hat(X[:,list(efficient_attribute_indices)], Y)
    print_str = f"此时的多元回归方程为：sales = {beta[0]:.6f} "
    for i in range(1, len(efficient_attribute_indices)):
        print_str += f'+ {beta[i]:.6f} * {df_data.columns[i]} '
    print(print_str)

多元回归的显著性检验F值 = 425.5208694395026，而在显著性水平0.05的情况下F(4, 195)为2.4179625418439827，所以该多元回归有显著性
此时的多元回归方程为：sales = 2.938889 + 0.045765 * TV + 0.188530 * radio + -0.001037 * newspaper 


In [9]:
# 测试sklearn包中的线性回归
from sklearn.linear_model import LinearRegression
reg = LinearRegression(fit_intercept=False).fit(X,Y)
reg.score(X, Y)

0.8972106381789522

In [10]:
reg.coef_
# 可以观察到，和手动算的结果一致。

array([ 2.93888937e+00,  4.57646455e-02,  1.88530017e-01, -1.03749304e-03])